# Compare Modified HRU Deltas

Two pathways are used within the framework.

1. H0: null hypothesis of historical weather statistics
2. H1: alternative hypothesis of weather statistics extracted from down-scaled, global climate model results.

There are four analysis periods

1. Data Period: 1981-2010
2. Projection Period 1: 2011-2040
3. Projection Period 2: 2041-2070
4. Projection Period 3: 2071-2100

## Imports and Parameters

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
from math import floor, ceil
from copy import deepcopy

In [3]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [4]:
# input directories and file roots
IN_DIR = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\NumModel\HSPF\DC_10k_Real'
OUT_DIR = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\NumModel\HSPF\DC_10k_Real\Processed'
OUT_ROOT = "DCmHBas_HRUs_"
NUMREAL = 10000
IN_RAET0 = "R%d_H0_AET_DF.pickle"
IN_RAET1 = "R%d_H1_AET_DF.pickle"
IN_RRE0 = "R%d_H0_Re_DF.pickle"
IN_RRE1 = "R%d_H1_Re_DF.pickle"
IN_RRO0 = "R%d_H0_RO_DF.pickle"
IN_RRO1 = "R%d_H1_RO_DF.pickle"

In [5]:
# time periods
TP_DICT = { 1 : [ dt.datetime(1981, 1, 1), dt.datetime(2010, 12, 31)],
            2 : [ dt.datetime(2011, 1, 1), dt.datetime(2040, 12, 31)],
            3 : [ dt.datetime(2041, 1, 1), dt.datetime(2070, 12, 31)],
            4 : [ dt.datetime(2071, 1, 1), dt.datetime(2100, 12, 31)],}

In [6]:
OurQs = np.array( [ round( 0.01 * x, 2 ) for x in range(101) ], dtype=np.float32 )
NumQs = len( OurQs )
NumQs

101

In [7]:
StatsHdsList = list()
for iI in range(NumQs):
    cQ = OurQs[iI]
    Hdr = "%d_ptile" % round(cQ * 100.0)
    StatsHdsList.append( Hdr )
# end of for
StatsHdsList.append( "Average" )
StatsHdsList.append( "Variance" )
StatsHdsList.append( "Skew" )
StatsHdsList.append( "Kurt" )

In [8]:
NumHdrs = len( StatsHdsList )
NumHdrs

105

In [9]:
HRU_Labels = [ "HRU_1", "HRU_2" ]
RR_Labels = [ "R001", "R002", "R003", "R004", "R005" ]

In [10]:
AET_In_Hdrs = ["HRU_1", 
               "HRU_2", 
               "R001", 
               "R002", 
               "R003", 
               "R004", 
               "R005" ]
RE_In_Hdrs = ["HRU_1", 
               "HRU_2", 
               "R001", 
               "R002", 
               "R003", 
               "R004" ]
RO_In_Hdrs = ["HRU_1", 
               "HRU_2", 
               "R001", 
               "R002", 
               "R003", 
               "R004", 
               "R005" ]

## Collate DataFrames

Go through all realizations and create a DataFrame for each constituent and each period

In [11]:
for rR in range( 1, NUMREAL + 1, 1 ):
    if ((rR % 200) == 0):
        print("Working on realization %d" % rR)
    # end if
    # AET
    InFile = IN_RAET0 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[AET_In_Hdrs].copy()
    H0AET = InDF.resample( 'MS', closed='left', label='left' ).sum()
    InFile = IN_RAET1 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[AET_In_Hdrs].copy()
    H1AET = InDF.resample( 'MS', closed='left', label='left' ).sum()
    DelAET = H1AET - H0AET
    # RO
    InFile = IN_RRO0 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[RO_In_Hdrs].copy()
    H0RO = InDF.resample( 'MS', closed='left', label='left' ).sum()
    InFile = IN_RRO1 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[RO_In_Hdrs].copy()
    H1RO = InDF.resample( 'MS', closed='left', label='left' ).sum()
    DelRO = H1RO - H0RO
    # Re
    InFile = IN_RRE0 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[RE_In_Hdrs].copy()
    H0RE = InDF.resample( 'MS', closed='left', label='left' ).sum()
    InFile = IN_RRE1 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[RE_In_Hdrs].copy()
    H1RE = InDF.resample( 'MS', closed='left', label='left' ).sum()
    DelRE = H1RE - H0RE
    if rR <= 1:
        #    HRU 1
        AET_HR1Del_1 = DelAET[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        AET_HR1Del_1["R_%d" % rR] = AET_HR1Del_1[HRU_Labels[0]].to_numpy()
        AET_HR1Del_1.drop( columns=[HRU_Labels[0]], inplace=True )
        RO_HR1Del_1 = DelRO[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        RO_HR1Del_1["R_%d" % rR] = RO_HR1Del_1[HRU_Labels[0]].to_numpy()
        RO_HR1Del_1.drop( columns=[HRU_Labels[0]], inplace=True )
        Re_HR1Del_1 = DelRE[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        Re_HR1Del_1["R_%d" % rR] = Re_HR1Del_1[HRU_Labels[0]].to_numpy()
        Re_HR1Del_1.drop( columns=[HRU_Labels[0]], inplace=True )
        AET_HR1Del_2 = DelAET[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        AET_HR1Del_2["R_%d" % rR] = AET_HR1Del_2[HRU_Labels[0]].to_numpy()
        AET_HR1Del_2.drop( columns=[HRU_Labels[0]], inplace=True )
        RO_HR1Del_2 = DelRO[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        RO_HR1Del_2["R_%d" % rR] = RO_HR1Del_2[HRU_Labels[0]].to_numpy()
        RO_HR1Del_2.drop( columns=[HRU_Labels[0]], inplace=True )
        Re_HR1Del_2 = DelRE[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        Re_HR1Del_2["R_%d" % rR] = Re_HR1Del_2[HRU_Labels[0]].to_numpy()
        Re_HR1Del_2.drop( columns=[HRU_Labels[0]], inplace=True )
        AET_HR1Del_3 = DelAET[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        AET_HR1Del_3["R_%d" % rR] = AET_HR1Del_3[HRU_Labels[0]].to_numpy()
        AET_HR1Del_3.drop( columns=[HRU_Labels[0]], inplace=True )
        RO_HR1Del_3 = DelRO[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        RO_HR1Del_3["R_%d" % rR] = RO_HR1Del_3[HRU_Labels[0]].to_numpy()
        RO_HR1Del_3.drop( columns=[HRU_Labels[0]], inplace=True )
        Re_HR1Del_3 = DelRE[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        Re_HR1Del_3["R_%d" % rR] = Re_HR1Del_3[HRU_Labels[0]].to_numpy()
        Re_HR1Del_3.drop( columns=[HRU_Labels[0]], inplace=True )
        AET_HR1Del_4 = DelAET[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        AET_HR1Del_4["R_%d" % rR] = AET_HR1Del_4[HRU_Labels[0]].to_numpy()
        AET_HR1Del_4.drop( columns=[HRU_Labels[0]], inplace=True )
        RO_HR1Del_4 = DelRO[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        RO_HR1Del_4["R_%d" % rR] = RO_HR1Del_4[HRU_Labels[0]].to_numpy()
        RO_HR1Del_4.drop( columns=[HRU_Labels[0]], inplace=True )
        Re_HR1Del_4 = DelRE[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        Re_HR1Del_4["R_%d" % rR] = Re_HR1Del_4[HRU_Labels[0]].to_numpy()
        Re_HR1Del_4.drop( columns=[HRU_Labels[0]], inplace=True )
        #    HRU 2
        AET_HR2Del_1 = DelAET[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        AET_HR2Del_1["R_%d" % rR] = AET_HR2Del_1[HRU_Labels[1]].to_numpy()
        AET_HR2Del_1.drop( columns=[HRU_Labels[1]], inplace=True )
        RO_HR2Del_1 = DelRO[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        RO_HR2Del_1["R_%d" % rR] = RO_HR2Del_1[HRU_Labels[1]].to_numpy()
        RO_HR2Del_1.drop( columns=[HRU_Labels[1]], inplace=True )
        Re_HR2Del_1 = DelRE[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        Re_HR2Del_1["R_%d" % rR] = Re_HR2Del_1[HRU_Labels[1]].to_numpy()
        Re_HR2Del_1.drop( columns=[HRU_Labels[1]], inplace=True )
        AET_HR2Del_2 = DelAET[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        AET_HR2Del_2["R_%d" % rR] = AET_HR2Del_2[HRU_Labels[1]].to_numpy()
        AET_HR2Del_2.drop( columns=[HRU_Labels[1]], inplace=True )
        RO_HR2Del_2 = DelRO[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        RO_HR2Del_2["R_%d" % rR] = RO_HR2Del_2[HRU_Labels[1]].to_numpy()
        RO_HR2Del_2.drop( columns=[HRU_Labels[1]], inplace=True )
        Re_HR2Del_2 = DelRE[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        Re_HR2Del_2["R_%d" % rR] = Re_HR2Del_2[HRU_Labels[1]].to_numpy()
        Re_HR2Del_2.drop( columns=[HRU_Labels[1]], inplace=True )
        AET_HR2Del_3 = DelAET[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        AET_HR2Del_3["R_%d" % rR] = AET_HR2Del_3[HRU_Labels[1]].to_numpy()
        AET_HR2Del_3.drop( columns=[HRU_Labels[1]], inplace=True )
        RO_HR2Del_3 = DelRO[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        RO_HR2Del_3["R_%d" % rR] = RO_HR2Del_3[HRU_Labels[1]].to_numpy()
        RO_HR2Del_3.drop( columns=[HRU_Labels[1]], inplace=True )
        Re_HR2Del_3 = DelRE[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        Re_HR2Del_3["R_%d" % rR] = Re_HR2Del_3[HRU_Labels[1]].to_numpy()
        Re_HR2Del_3.drop( columns=[HRU_Labels[1]], inplace=True )
        AET_HR2Del_4 = DelAET[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        AET_HR2Del_4["R_%d" % rR] = AET_HR2Del_4[HRU_Labels[1]].to_numpy()
        AET_HR2Del_4.drop( columns=[HRU_Labels[1]], inplace=True )
        RO_HR2Del_4 = DelRO[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        RO_HR2Del_4["R_%d" % rR] = RO_HR2Del_4[HRU_Labels[1]].to_numpy()
        RO_HR2Del_4.drop( columns=[HRU_Labels[1]], inplace=True )
        Re_HR2Del_4 = DelRE[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        Re_HR2Del_4["R_%d" % rR] = Re_HR2Del_4[HRU_Labels[1]].to_numpy()
        Re_HR2Del_4.drop( columns=[HRU_Labels[1]], inplace=True )
        #    RR 1
        AET_RR1Del_1 = DelAET[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        AET_RR1Del_1["R_%d" % rR] = AET_RR1Del_1[RR_Labels[0]].to_numpy()
        AET_RR1Del_1.drop( columns=[RR_Labels[0]], inplace=True )
        RO_RR1Del_1 = DelRO[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        RO_RR1Del_1["R_%d" % rR] = RO_RR1Del_1[RR_Labels[0]].to_numpy()
        RO_RR1Del_1.drop( columns=[RR_Labels[0]], inplace=True )
        Re_RR1Del_1 = DelRE[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        Re_RR1Del_1["R_%d" % rR] = Re_RR1Del_1[RR_Labels[0]].to_numpy()
        Re_RR1Del_1.drop( columns=[RR_Labels[0]], inplace=True )
        AET_RR1Del_2 = DelAET[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        AET_RR1Del_2["R_%d" % rR] = AET_RR1Del_2[RR_Labels[0]].to_numpy()
        AET_RR1Del_2.drop( columns=[RR_Labels[0]], inplace=True )
        RO_RR1Del_2 = DelRO[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        RO_RR1Del_2["R_%d" % rR] = RO_RR1Del_2[RR_Labels[0]].to_numpy()
        RO_RR1Del_2.drop( columns=[RR_Labels[0]], inplace=True )
        Re_RR1Del_2 = DelRE[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        Re_RR1Del_2["R_%d" % rR] = Re_RR1Del_2[RR_Labels[0]].to_numpy()
        Re_RR1Del_2.drop( columns=[RR_Labels[0]], inplace=True )
        AET_RR1Del_3 = DelAET[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        AET_RR1Del_3["R_%d" % rR] = AET_RR1Del_3[RR_Labels[0]].to_numpy()
        AET_RR1Del_3.drop( columns=[RR_Labels[0]], inplace=True )
        RO_RR1Del_3 = DelRO[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        RO_RR1Del_3["R_%d" % rR] = RO_RR1Del_3[RR_Labels[0]].to_numpy()
        RO_RR1Del_3.drop( columns=[RR_Labels[0]], inplace=True )
        Re_RR1Del_3 = DelRE[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        Re_RR1Del_3["R_%d" % rR] = Re_RR1Del_3[RR_Labels[0]].to_numpy()
        Re_RR1Del_3.drop( columns=[RR_Labels[0]], inplace=True )
        AET_RR1Del_4 = DelAET[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        AET_RR1Del_4["R_%d" % rR] = AET_RR1Del_4[RR_Labels[0]].to_numpy()
        AET_RR1Del_4.drop( columns=[RR_Labels[0]], inplace=True )
        RO_RR1Del_4 = DelRO[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        RO_RR1Del_4["R_%d" % rR] = RO_RR1Del_4[RR_Labels[0]].to_numpy()
        RO_RR1Del_4.drop( columns=[RR_Labels[0]], inplace=True )
        Re_RR1Del_4 = DelRE[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        Re_RR1Del_4["R_%d" % rR] = Re_RR1Del_4[RR_Labels[0]].to_numpy()
        Re_RR1Del_4.drop( columns=[RR_Labels[0]], inplace=True )
        #    RR 2
        AET_RR2Del_1 = DelAET[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        AET_RR2Del_1["R_%d" % rR] = AET_RR2Del_1[RR_Labels[1]].to_numpy()
        AET_RR2Del_1.drop( columns=[RR_Labels[1]], inplace=True )
        RO_RR2Del_1 = DelRO[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        RO_RR2Del_1["R_%d" % rR] = RO_RR2Del_1[RR_Labels[1]].to_numpy()
        RO_RR2Del_1.drop( columns=[RR_Labels[1]], inplace=True )
        Re_RR2Del_1 = DelRE[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        Re_RR2Del_1["R_%d" % rR] = Re_RR2Del_1[RR_Labels[1]].to_numpy()
        Re_RR2Del_1.drop( columns=[RR_Labels[1]], inplace=True )
        AET_RR2Del_2 = DelAET[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        AET_RR2Del_2["R_%d" % rR] = AET_RR2Del_2[RR_Labels[1]].to_numpy()
        AET_RR2Del_2.drop( columns=[RR_Labels[1]], inplace=True )
        RO_RR2Del_2 = DelRO[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        RO_RR2Del_2["R_%d" % rR] = RO_RR2Del_2[RR_Labels[1]].to_numpy()
        RO_RR2Del_2.drop( columns=[RR_Labels[1]], inplace=True )
        Re_RR2Del_2 = DelRE[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        Re_RR2Del_2["R_%d" % rR] = Re_RR2Del_2[RR_Labels[1]].to_numpy()
        Re_RR2Del_2.drop( columns=[RR_Labels[1]], inplace=True )
        AET_RR2Del_3 = DelAET[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        AET_RR2Del_3["R_%d" % rR] = AET_RR2Del_3[RR_Labels[1]].to_numpy()
        AET_RR2Del_3.drop( columns=[RR_Labels[1]], inplace=True )
        RO_RR2Del_3 = DelRO[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        RO_RR2Del_3["R_%d" % rR] = RO_RR2Del_3[RR_Labels[1]].to_numpy()
        RO_RR2Del_3.drop( columns=[RR_Labels[1]], inplace=True )
        Re_RR2Del_3 = DelRE[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        Re_RR2Del_3["R_%d" % rR] = Re_RR2Del_3[RR_Labels[1]].to_numpy()
        Re_RR2Del_3.drop( columns=[RR_Labels[1]], inplace=True )
        AET_RR2Del_4 = DelAET[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        AET_RR2Del_4["R_%d" % rR] = AET_RR2Del_4[RR_Labels[1]].to_numpy()
        AET_RR2Del_4.drop( columns=[RR_Labels[1]], inplace=True )
        RO_RR2Del_4 = DelRO[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        RO_RR2Del_4["R_%d" % rR] = RO_RR2Del_4[RR_Labels[1]].to_numpy()
        RO_RR2Del_4.drop( columns=[RR_Labels[1]], inplace=True )
        Re_RR2Del_4 = DelRE[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        Re_RR2Del_4["R_%d" % rR] = Re_RR2Del_4[RR_Labels[1]].to_numpy()
        Re_RR2Del_4.drop( columns=[RR_Labels[1]], inplace=True )
        #    RR 3
        AET_RR3Del_1 = DelAET[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        AET_RR3Del_1["R_%d" % rR] = AET_RR3Del_1[RR_Labels[2]].to_numpy()
        AET_RR3Del_1.drop( columns=[RR_Labels[2]], inplace=True )
        RO_RR3Del_1 = DelRO[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        RO_RR3Del_1["R_%d" % rR] = RO_RR3Del_1[RR_Labels[2]].to_numpy()
        RO_RR3Del_1.drop( columns=[RR_Labels[2]], inplace=True )
        Re_RR3Del_1 = DelRE[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        Re_RR3Del_1["R_%d" % rR] = Re_RR3Del_1[RR_Labels[2]].to_numpy()
        Re_RR3Del_1.drop( columns=[RR_Labels[2]], inplace=True )
        AET_RR3Del_2 = DelAET[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        AET_RR3Del_2["R_%d" % rR] = AET_RR3Del_2[RR_Labels[2]].to_numpy()
        AET_RR3Del_2.drop( columns=[RR_Labels[2]], inplace=True )
        RO_RR3Del_2 = DelRO[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        RO_RR3Del_2["R_%d" % rR] = RO_RR3Del_2[RR_Labels[2]].to_numpy()
        RO_RR3Del_2.drop( columns=[RR_Labels[2]], inplace=True )
        Re_RR3Del_2 = DelRE[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        Re_RR3Del_2["R_%d" % rR] = Re_RR3Del_2[RR_Labels[2]].to_numpy()
        Re_RR3Del_2.drop( columns=[RR_Labels[2]], inplace=True )
        AET_RR3Del_3 = DelAET[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        AET_RR3Del_3["R_%d" % rR] = AET_RR3Del_3[RR_Labels[2]].to_numpy()
        AET_RR3Del_3.drop( columns=[RR_Labels[2]], inplace=True )
        RO_RR3Del_3 = DelRO[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        RO_RR3Del_3["R_%d" % rR] = RO_RR3Del_3[RR_Labels[2]].to_numpy()
        RO_RR3Del_3.drop( columns=[RR_Labels[2]], inplace=True )
        Re_RR3Del_3 = DelRE[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        Re_RR3Del_3["R_%d" % rR] = Re_RR3Del_3[RR_Labels[2]].to_numpy()
        Re_RR3Del_3.drop( columns=[RR_Labels[2]], inplace=True )
        AET_RR3Del_4 = DelAET[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        AET_RR3Del_4["R_%d" % rR] = AET_RR3Del_4[RR_Labels[2]].to_numpy()
        AET_RR3Del_4.drop( columns=[RR_Labels[2]], inplace=True )
        RO_RR3Del_4 = DelRO[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        RO_RR3Del_4["R_%d" % rR] = RO_RR3Del_4[RR_Labels[2]].to_numpy()
        RO_RR3Del_4.drop( columns=[RR_Labels[2]], inplace=True )
        Re_RR3Del_4 = DelRE[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        Re_RR3Del_4["R_%d" % rR] = Re_RR3Del_4[RR_Labels[2]].to_numpy()
        Re_RR3Del_4.drop( columns=[RR_Labels[2]], inplace=True )
        #    RR 4
        AET_RR4Del_1 = DelAET[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        AET_RR4Del_1["R_%d" % rR] = AET_RR4Del_1[RR_Labels[3]].to_numpy()
        AET_RR4Del_1.drop( columns=[RR_Labels[3]], inplace=True )
        RO_RR4Del_1 = DelRO[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        RO_RR4Del_1["R_%d" % rR] = RO_RR4Del_1[RR_Labels[3]].to_numpy()
        RO_RR4Del_1.drop( columns=[RR_Labels[3]], inplace=True )
        Re_RR4Del_1 = DelRE[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        Re_RR4Del_1["R_%d" % rR] = Re_RR4Del_1[RR_Labels[3]].to_numpy()
        Re_RR4Del_1.drop( columns=[RR_Labels[3]], inplace=True )
        AET_RR4Del_2 = DelAET[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        AET_RR4Del_2["R_%d" % rR] = AET_RR4Del_2[RR_Labels[3]].to_numpy()
        AET_RR4Del_2.drop( columns=[RR_Labels[3]], inplace=True )
        RO_RR4Del_2 = DelRO[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        RO_RR4Del_2["R_%d" % rR] = RO_RR4Del_2[RR_Labels[3]].to_numpy()
        RO_RR4Del_2.drop( columns=[RR_Labels[3]], inplace=True )
        Re_RR4Del_2 = DelRE[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        Re_RR4Del_2["R_%d" % rR] = Re_RR4Del_2[RR_Labels[3]].to_numpy()
        Re_RR4Del_2.drop( columns=[RR_Labels[3]], inplace=True )
        AET_RR4Del_3 = DelAET[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        AET_RR4Del_3["R_%d" % rR] = AET_RR4Del_3[RR_Labels[3]].to_numpy()
        AET_RR4Del_3.drop( columns=[RR_Labels[3]], inplace=True )
        RO_RR4Del_3 = DelRO[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        RO_RR4Del_3["R_%d" % rR] = RO_RR4Del_3[RR_Labels[3]].to_numpy()
        RO_RR4Del_3.drop( columns=[RR_Labels[3]], inplace=True )
        Re_RR4Del_3 = DelRE[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        Re_RR4Del_3["R_%d" % rR] = Re_RR4Del_3[RR_Labels[3]].to_numpy()
        Re_RR4Del_3.drop( columns=[RR_Labels[3]], inplace=True )
        AET_RR4Del_4 = DelAET[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        AET_RR4Del_4["R_%d" % rR] = AET_RR4Del_4[RR_Labels[3]].to_numpy()
        AET_RR4Del_4.drop( columns=[RR_Labels[3]], inplace=True )
        RO_RR4Del_4 = DelRO[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        RO_RR4Del_4["R_%d" % rR] = RO_RR4Del_4[RR_Labels[3]].to_numpy()
        RO_RR4Del_4.drop( columns=[RR_Labels[3]], inplace=True )
        Re_RR4Del_4 = DelRE[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        Re_RR4Del_4["R_%d" % rR] = Re_RR4Del_4[RR_Labels[3]].to_numpy()
        Re_RR4Del_4.drop( columns=[RR_Labels[3]], inplace=True )
        #    RR 5
        AET_RR5Del_1 = DelAET[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        AET_RR5Del_1["R_%d" % rR] = AET_RR5Del_1[RR_Labels[4]].to_numpy()
        AET_RR5Del_1.drop( columns=[RR_Labels[4]], inplace=True )
        RO_RR5Del_1 = DelRO[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        RO_RR5Del_1["R_%d" % rR] = RO_RR5Del_1[RR_Labels[4]].to_numpy()
        RO_RR5Del_1.drop( columns=[RR_Labels[4]], inplace=True )
        AET_RR5Del_2 = DelAET[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        AET_RR5Del_2["R_%d" % rR] = AET_RR5Del_2[RR_Labels[4]].to_numpy()
        AET_RR5Del_2.drop( columns=[RR_Labels[4]], inplace=True )
        RO_RR5Del_2 = DelRO[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        RO_RR5Del_2["R_%d" % rR] = RO_RR5Del_2[RR_Labels[4]].to_numpy()
        RO_RR5Del_2.drop( columns=[RR_Labels[4]], inplace=True )
        AET_RR5Del_3 = DelAET[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        AET_RR5Del_3["R_%d" % rR] = AET_RR5Del_3[RR_Labels[4]].to_numpy()
        AET_RR5Del_3.drop( columns=[RR_Labels[4]], inplace=True )
        RO_RR5Del_3 = DelRO[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        RO_RR5Del_3["R_%d" % rR] = RO_RR5Del_3[RR_Labels[4]].to_numpy()
        RO_RR5Del_3.drop( columns=[RR_Labels[4]], inplace=True )
        AET_RR5Del_4 = DelAET[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        AET_RR5Del_4["R_%d" % rR] = AET_RR5Del_4[RR_Labels[4]].to_numpy()
        AET_RR5Del_4.drop( columns=[RR_Labels[4]], inplace=True )
        RO_RR5Del_4 = DelRO[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        RO_RR5Del_4["R_%d" % rR] = RO_RR5Del_4[RR_Labels[4]].to_numpy()
        RO_RR5Del_4.drop( columns=[RR_Labels[4]], inplace=True )
    else:
        # HRU 1
        tmpAET = DelAET[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].to_numpy()
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        AET_HR1Del_1 = AET_HR1Del_1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].to_numpy()
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        RO_HR1Del_1 = RO_HR1Del_1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[HRU_Labels[0]].to_numpy()
        tmpRe.drop( columns=[HRU_Labels[0]], inplace=True )
        Re_HR1Del_1 = Re_HR1Del_1.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].to_numpy()
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        AET_HR1Del_2 = AET_HR1Del_2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].to_numpy()
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        RO_HR1Del_2 = RO_HR1Del_2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[HRU_Labels[0]].to_numpy()
        tmpRe.drop( columns=[HRU_Labels[0]], inplace=True )
        Re_HR1Del_2 = Re_HR1Del_2.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].to_numpy()
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        AET_HR1Del_3= AET_HR1Del_3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].to_numpy()
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        RO_HR1Del_3 = RO_HR1Del_3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[HRU_Labels[0]].to_numpy()
        tmpRe.drop( columns=[HRU_Labels[0]], inplace=True )
        Re_HR1Del_3 = Re_HR1Del_3.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].to_numpy()
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        AET_HR1Del_4 = AET_HR1Del_4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].to_numpy()
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        RO_HR1Del_4 = RO_HR1Del_4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[HRU_Labels[0]].to_numpy()
        tmpRe.drop( columns=[HRU_Labels[0]], inplace=True )
        Re_HR1Del_4 = Re_HR1Del_4.merge( tmpRe, how='inner', left_index=True, right_index=True )
        # HRU 2
        tmpAET = DelAET[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].to_numpy()
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        AET_HR2Del_1 = AET_HR2Del_1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].to_numpy()
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        RO_HR2Del_1 = RO_HR2Del_1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[HRU_Labels[1]].to_numpy()
        tmpRe.drop( columns=[HRU_Labels[1]], inplace=True )
        Re_HR2Del_1 = Re_HR2Del_1.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].to_numpy()
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        AET_HR2Del_2 = AET_HR2Del_2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].to_numpy()
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        RO_HR2Del_2 = RO_HR2Del_2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[HRU_Labels[1]].to_numpy()
        tmpRe.drop( columns=[HRU_Labels[1]], inplace=True )
        Re_HR2Del_2 = Re_HR2Del_2.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].to_numpy()
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        AET_HR2Del_3= AET_HR2Del_3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].to_numpy()
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        RO_HR2Del_3 = RO_HR2Del_3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[HRU_Labels[1]].to_numpy()
        tmpRe.drop( columns=[HRU_Labels[1]], inplace=True )
        Re_HR2Del_3 = Re_HR2Del_3.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].to_numpy()
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        AET_HR2Del_4 = AET_HR2Del_4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].to_numpy()
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        RO_HR2Del_4 = RO_HR2Del_4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[HRU_Labels[1]].to_numpy()
        tmpRe.drop( columns=[HRU_Labels[1]], inplace=True )
        Re_HR2Del_4 = Re_HR2Del_4.merge( tmpRe, how='inner', left_index=True, right_index=True )
        # RR 1
        tmpAET = DelAET[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        AET_RR1Del_1 = AET_RR1Del_1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        RO_RR1Del_1 = RO_RR1Del_1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[0]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[0]], inplace=True )
        Re_RR1Del_1 = Re_RR1Del_1.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        AET_RR1Del_2 = AET_RR1Del_2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        RO_RR1Del_2 = RO_RR1Del_2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[0]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[0]], inplace=True )
        Re_RR1Del_2 = Re_RR1Del_2.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        AET_RR1Del_3 = AET_RR1Del_3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        RO_RR1Del_3 = RO_RR1Del_3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[0]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[0]], inplace=True )
        Re_RR1Del_3 = Re_RR1Del_3.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        AET_RR1Del_4 = AET_RR1Del_4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        RO_RR1Del_4 = RO_RR1Del_4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[0]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[0]], inplace=True )
        Re_RR1Del_4 = Re_RR1Del_4.merge( tmpRe, how='inner', left_index=True, right_index=True )
        # RR 2
        tmpAET = DelAET[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        AET_RR2Del_1 = AET_RR2Del_1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        RO_RR2Del_1 = RO_RR2Del_1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[1]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[1]], inplace=True )
        Re_RR2Del_1 = Re_RR2Del_1.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        AET_RR2Del_2 = AET_RR2Del_2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        RO_RR2Del_2 = RO_RR2Del_2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[1]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[1]], inplace=True )
        Re_RR2Del_2 = Re_RR2Del_2.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        AET_RR2Del_3 = AET_RR2Del_3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        RO_RR2Del_3 = RO_RR2Del_3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[1]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[1]], inplace=True )
        Re_RR2Del_3 = Re_RR2Del_3.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        AET_RR2Del_4 = AET_RR2Del_4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        RO_RR2Del_4 = RO_RR2Del_4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[1]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[1]], inplace=True )
        Re_RR2Del_4 = Re_RR2Del_4.merge( tmpRe, how='inner', left_index=True, right_index=True )
        # RR 3
        tmpAET = DelAET[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        AET_RR3Del_1 = AET_RR3Del_1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        RO_RR3Del_1 = RO_RR3Del_1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[2]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[2]], inplace=True )
        Re_RR3Del_1 = Re_RR3Del_1.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        AET_RR3Del_2 = AET_RR3Del_2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        RO_RR3Del_2 = RO_RR3Del_2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[2]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[2]], inplace=True )
        Re_RR3Del_2 = Re_RR3Del_2.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        AET_RR3Del_3 = AET_RR3Del_3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        RO_RR3Del_3 = RO_RR3Del_3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[2]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[2]], inplace=True )
        Re_RR3Del_3 = Re_RR3Del_3.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        AET_RR3Del_4 = AET_RR3Del_4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        RO_RR3Del_4 = RO_RR3Del_4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[2]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[2]], inplace=True )
        Re_RR3Del_4 = Re_RR3Del_4.merge( tmpRe, how='inner', left_index=True, right_index=True )
        # RR 4
        tmpAET = DelAET[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        AET_RR4Del_1 = AET_RR4Del_1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        RO_RR4Del_1 = RO_RR4Del_1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[3]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[3]], inplace=True )
        Re_RR4Del_1 = Re_RR4Del_1.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        AET_RR4Del_2 = AET_RR4Del_2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        RO_RR4Del_2 = RO_RR4Del_2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[3]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[3]], inplace=True )
        Re_RR4Del_2 = Re_RR4Del_2.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        AET_RR4Del_3 = AET_RR4Del_3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        RO_RR4Del_3 = RO_RR4Del_3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[3]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[3]], inplace=True )
        Re_RR4Del_3 = Re_RR4Del_3.merge( tmpRe, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        AET_RR4Del_4 = AET_RR4Del_4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        RO_RR4Del_4 = RO_RR4Del_4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRe = DelRE[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRe["R_%d" % rR] = tmpRe[RR_Labels[3]].to_numpy()
        tmpRe.drop( columns=[RR_Labels[3]], inplace=True )
        Re_RR4Del_4 = Re_RR4Del_4.merge( tmpRe, how='inner', left_index=True, right_index=True )
        # RR 5
        tmpAET = DelAET[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        AET_RR5Del_1 = AET_RR5Del_1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        RO_RR5Del_1 = RO_RR5Del_1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        AET_RR5Del_2 = AET_RR5Del_2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        RO_RR5Del_2 = RO_RR5Del_2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        AET_RR5Del_3 = AET_RR5Del_3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        RO_RR5Del_3 = RO_RR5Del_3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpAET = DelAET[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].to_numpy()
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        AET_RR5Del_4 = AET_RR5Del_4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = DelRO[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].to_numpy()
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        RO_RR5Del_4 = RO_RR5Del_4.merge( tmpRO, how='inner', left_index=True, right_index=True )
    # end if
# end for

Working on realization 200
Working on realization 400
Working on realization 600
Working on realization 800
Working on realization 1000
Working on realization 1200
Working on realization 1400
Working on realization 1600
Working on realization 1800
Working on realization 2000
Working on realization 2200
Working on realization 2400
Working on realization 2600
Working on realization 2800
Working on realization 3000
Working on realization 3200
Working on realization 3400
Working on realization 3600
Working on realization 3800
Working on realization 4000
Working on realization 4200
Working on realization 4400
Working on realization 4600
Working on realization 4800
Working on realization 5000
Working on realization 5200
Working on realization 5400
Working on realization 5600
Working on realization 5800
Working on realization 6000
Working on realization 6200
Working on realization 6400
Working on realization 6600
Working on realization 6800
Working on realization 7000
Working on realization 7

This loop takes a long time so output this files to pickles now

In [12]:
OFNamer = "Del_HRU1_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_HR1Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU1_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_HR1Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU1_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_HR1Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU1_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_HR1Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_HR2Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_HR2Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_HR2Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_HR2Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR1Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR1Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR1Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR1Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR2Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR2Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR2Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR2Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR3Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR3Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR3Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR3Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR4Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR4Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR4Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR4Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR5_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR5Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR5_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR5Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR5_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR5Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR5_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
AET_RR5Del_4.to_pickle( OFPath, compression='zip' )

In [13]:
OFNamer = "Del_HRU1_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_HR1Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU1_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_HR1Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU1_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_HR1Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU1_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_HR1Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_HR2Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_HR2Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_HR2Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_HR2Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR1Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR1Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR1Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR1Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR2Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR2Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR2Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR2Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR3Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR3Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR3Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR3Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR4Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR4Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR4Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR4Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR5_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR5Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR5_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR5Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR5_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR5Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR5_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
RO_RR5Del_4.to_pickle( OFPath, compression='zip' )

In [14]:
OFNamer = "Del_HRU1_Re1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_HR1Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU1_Re2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_HR1Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU1_Re3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_HR1Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU1_Re4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_HR1Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_Re1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_HR2Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_Re2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_HR2Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_Re3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_HR2Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_HRU2_Re4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_HR2Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_Re1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR1Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_Re2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR1Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_Re3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR1Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR1_Re4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR1Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_Re1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR2Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_Re2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR2Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_Re3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR2Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR2_Re4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR2Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_Re1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR3Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_Re2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR3Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_Re3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR3Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR3_Re4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR3Del_4.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_Re1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR4Del_1.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_Re2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR4Del_2.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_Re3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR4Del_3.to_pickle( OFPath, compression='zip' )
OFNamer = "Del_RR4_Re4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
Re_RR4Del_4.to_pickle( OFPath, compression='zip' )

## Calculate Time History Stats

Calculate the time history stats now for each DataFrame

In [15]:
DictHR1AET1 = dict()
DictHR1AET2 = dict()
DictHR1AET3 = dict()
DictHR1AET4 = dict()
DictHR2AET1 = dict()
DictHR2AET2 = dict()
DictHR2AET3 = dict()
DictHR2AET4 = dict()
DictRR1AET1 = dict()
DictRR1AET2 = dict()
DictRR1AET3 = dict()
DictRR1AET4 = dict()
DictRR2AET1 = dict()
DictRR2AET2 = dict()
DictRR2AET3 = dict()
DictRR2AET4 = dict()
DictRR3AET1 = dict()
DictRR3AET2 = dict()
DictRR3AET3 = dict()
DictRR3AET4 = dict()
DictRR4AET1 = dict()
DictRR4AET2 = dict()
DictRR4AET3 = dict()
DictRR4AET4 = dict()
DictRR5AET1 = dict()
DictRR5AET2 = dict()
DictRR5AET3 = dict()
DictRR5AET4 = dict()
DictHR1RO1 = dict()
DictHR1RO2 = dict()
DictHR1RO3 = dict()
DictHR1RO4 = dict()
DictHR2RO1 = dict()
DictHR2RO2 = dict()
DictHR2RO3 = dict()
DictHR2RO4 = dict()
DictRR1RO1 = dict()
DictRR1RO2 = dict()
DictRR1RO3 = dict()
DictRR1RO4 = dict()
DictRR2RO1 = dict()
DictRR2RO2 = dict()
DictRR2RO3 = dict()
DictRR2RO4 = dict()
DictRR3RO1 = dict()
DictRR3RO2 = dict()
DictRR3RO3 = dict()
DictRR3RO4 = dict()
DictRR4RO1 = dict()
DictRR4RO2 = dict()
DictRR4RO3 = dict()
DictRR4RO4 = dict()
DictRR5RO1 = dict()
DictRR5RO2 = dict()
DictRR5RO3 = dict()
DictRR5RO4 = dict()
DictHR1RE1 = dict()
DictHR1RE2 = dict()
DictHR1RE3 = dict()
DictHR1RE4 = dict()
DictHR2RE1 = dict()
DictHR2RE2 = dict()
DictHR2RE3 = dict()
DictHR2RE4 = dict()
DictRR1RE1 = dict()
DictRR1RE2 = dict()
DictRR1RE3 = dict()
DictRR1RE4 = dict()
DictRR2RE1 = dict()
DictRR2RE2 = dict()
DictRR2RE3 = dict()
DictRR2RE4 = dict()
DictRR3RE1 = dict()
DictRR3RE2 = dict()
DictRR3RE3 = dict()
DictRR3RE4 = dict()
DictRR4RE1 = dict()
DictRR4RE2 = dict()
DictRR4RE3 = dict()
DictRR4RE4 = dict()
hdrCnt = 0
# start out by doing the percentiles
for iI in range(NumQs):
    curQ = OurQs[iI]
    DictHR1AET1[StatsHdsList[hdrCnt]] = AET_HR1Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1AET2[StatsHdsList[hdrCnt]] = AET_HR1Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1AET3[StatsHdsList[hdrCnt]] = AET_HR1Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1AET4[StatsHdsList[hdrCnt]] = AET_HR1Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2AET1[StatsHdsList[hdrCnt]] = AET_HR2Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2AET2[StatsHdsList[hdrCnt]] = AET_HR2Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2AET3[StatsHdsList[hdrCnt]] = AET_HR2Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2AET4[StatsHdsList[hdrCnt]] = AET_HR2Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1AET1[StatsHdsList[hdrCnt]] = AET_RR1Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1AET2[StatsHdsList[hdrCnt]] = AET_RR1Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1AET3[StatsHdsList[hdrCnt]] = AET_RR1Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1AET4[StatsHdsList[hdrCnt]] = AET_RR1Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2AET1[StatsHdsList[hdrCnt]] = AET_RR2Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2AET2[StatsHdsList[hdrCnt]] = AET_RR2Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2AET3[StatsHdsList[hdrCnt]] = AET_RR2Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2AET4[StatsHdsList[hdrCnt]] = AET_RR2Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3AET1[StatsHdsList[hdrCnt]] = AET_RR3Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3AET2[StatsHdsList[hdrCnt]] = AET_RR3Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3AET3[StatsHdsList[hdrCnt]] = AET_RR3Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3AET4[StatsHdsList[hdrCnt]] = AET_RR3Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4AET1[StatsHdsList[hdrCnt]] = AET_RR4Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4AET2[StatsHdsList[hdrCnt]] = AET_RR4Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4AET3[StatsHdsList[hdrCnt]] = AET_RR4Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4AET4[StatsHdsList[hdrCnt]] = AET_RR4Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR5AET1[StatsHdsList[hdrCnt]] = AET_RR5Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR5AET2[StatsHdsList[hdrCnt]] = AET_RR5Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR5AET3[StatsHdsList[hdrCnt]] = AET_RR5Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR5AET4[StatsHdsList[hdrCnt]] = AET_RR5Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1RO1[StatsHdsList[hdrCnt]] = RO_HR1Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1RO2[StatsHdsList[hdrCnt]] = RO_HR1Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1RO3[StatsHdsList[hdrCnt]] = RO_HR1Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1RO4[StatsHdsList[hdrCnt]] = RO_HR1Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2RO1[StatsHdsList[hdrCnt]] = RO_HR2Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2RO2[StatsHdsList[hdrCnt]] = RO_HR2Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2RO3[StatsHdsList[hdrCnt]] = RO_HR2Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2RO4[StatsHdsList[hdrCnt]] = RO_HR2Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1RO1[StatsHdsList[hdrCnt]] = RO_RR1Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1RO2[StatsHdsList[hdrCnt]] = RO_RR1Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1RO3[StatsHdsList[hdrCnt]] = RO_RR1Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1RO4[StatsHdsList[hdrCnt]] = RO_RR1Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2RO1[StatsHdsList[hdrCnt]] = RO_RR2Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2RO2[StatsHdsList[hdrCnt]] = RO_RR2Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2RO3[StatsHdsList[hdrCnt]] = RO_RR2Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2RO4[StatsHdsList[hdrCnt]] = RO_RR2Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3RO1[StatsHdsList[hdrCnt]] = RO_RR3Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3RO2[StatsHdsList[hdrCnt]] = RO_RR3Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3RO3[StatsHdsList[hdrCnt]] = RO_RR3Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3RO4[StatsHdsList[hdrCnt]] = RO_RR3Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4RO1[StatsHdsList[hdrCnt]] = RO_RR4Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4RO2[StatsHdsList[hdrCnt]] = RO_RR4Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4RO3[StatsHdsList[hdrCnt]] = RO_RR4Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4RO4[StatsHdsList[hdrCnt]] = RO_RR4Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR5RO1[StatsHdsList[hdrCnt]] = RO_RR5Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR5RO2[StatsHdsList[hdrCnt]] = RO_RR5Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR5RO3[StatsHdsList[hdrCnt]] = RO_RR5Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR5RO4[StatsHdsList[hdrCnt]] = RO_RR5Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1RE1[StatsHdsList[hdrCnt]] = Re_HR1Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1RE2[StatsHdsList[hdrCnt]] = Re_HR1Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1RE3[StatsHdsList[hdrCnt]] = Re_HR1Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR1RE4[StatsHdsList[hdrCnt]] = Re_HR1Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2RE1[StatsHdsList[hdrCnt]] = Re_HR2Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2RE2[StatsHdsList[hdrCnt]] = Re_HR2Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2RE3[StatsHdsList[hdrCnt]] = Re_HR2Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictHR2RE4[StatsHdsList[hdrCnt]] = Re_HR2Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1RE1[StatsHdsList[hdrCnt]] = Re_RR1Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1RE2[StatsHdsList[hdrCnt]] = Re_RR1Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1RE3[StatsHdsList[hdrCnt]] = Re_RR1Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR1RE4[StatsHdsList[hdrCnt]] = Re_RR1Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2RE1[StatsHdsList[hdrCnt]] = Re_RR2Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2RE2[StatsHdsList[hdrCnt]] = Re_RR2Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2RE3[StatsHdsList[hdrCnt]] = Re_RR2Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR2RE4[StatsHdsList[hdrCnt]] = Re_RR2Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3RE1[StatsHdsList[hdrCnt]] = Re_RR3Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3RE2[StatsHdsList[hdrCnt]] = Re_RR3Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3RE3[StatsHdsList[hdrCnt]] = Re_RR3Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR3RE4[StatsHdsList[hdrCnt]] = Re_RR3Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4RE1[StatsHdsList[hdrCnt]] = Re_RR4Del_1.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4RE2[StatsHdsList[hdrCnt]] = Re_RR4Del_2.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4RE3[StatsHdsList[hdrCnt]] = Re_RR4Del_3.quantile( q=curQ, axis=1 ).to_numpy()
    DictRR4RE4[StatsHdsList[hdrCnt]] = Re_RR4Del_4.quantile( q=curQ, axis=1 ).to_numpy()
    hdrCnt += 1
# end for
DictHR1AET1[StatsHdsList[hdrCnt]] = AET_HR1Del_1.mean( axis=1 ).to_numpy()
DictHR1AET2[StatsHdsList[hdrCnt]] = AET_HR1Del_2.mean( axis=1 ).to_numpy()
DictHR1AET3[StatsHdsList[hdrCnt]] = AET_HR1Del_3.mean( axis=1 ).to_numpy()
DictHR1AET4[StatsHdsList[hdrCnt]] = AET_HR1Del_4.mean( axis=1 ).to_numpy()
DictHR2AET1[StatsHdsList[hdrCnt]] = AET_HR2Del_1.mean( axis=1 ).to_numpy()
DictHR2AET2[StatsHdsList[hdrCnt]] = AET_HR2Del_2.mean( axis=1 ).to_numpy()
DictHR2AET3[StatsHdsList[hdrCnt]] = AET_HR2Del_3.mean( axis=1 ).to_numpy()
DictHR2AET4[StatsHdsList[hdrCnt]] = AET_HR2Del_4.mean( axis=1 ).to_numpy()
DictRR1AET1[StatsHdsList[hdrCnt]] = AET_RR1Del_1.mean( axis=1 ).to_numpy()
DictRR1AET2[StatsHdsList[hdrCnt]] = AET_RR1Del_2.mean( axis=1 ).to_numpy()
DictRR1AET3[StatsHdsList[hdrCnt]] = AET_RR1Del_3.mean( axis=1 ).to_numpy()
DictRR1AET4[StatsHdsList[hdrCnt]] = AET_RR1Del_4.mean( axis=1 ).to_numpy()
DictRR2AET1[StatsHdsList[hdrCnt]] = AET_RR2Del_1.mean( axis=1 ).to_numpy()
DictRR2AET2[StatsHdsList[hdrCnt]] = AET_RR2Del_2.mean( axis=1 ).to_numpy()
DictRR2AET3[StatsHdsList[hdrCnt]] = AET_RR2Del_3.mean( axis=1 ).to_numpy()
DictRR2AET4[StatsHdsList[hdrCnt]] = AET_RR2Del_4.mean( axis=1 ).to_numpy()
DictRR3AET1[StatsHdsList[hdrCnt]] = AET_RR3Del_1.mean( axis=1 ).to_numpy()
DictRR3AET2[StatsHdsList[hdrCnt]] = AET_RR3Del_2.mean( axis=1 ).to_numpy()
DictRR3AET3[StatsHdsList[hdrCnt]] = AET_RR3Del_3.mean( axis=1 ).to_numpy()
DictRR3AET4[StatsHdsList[hdrCnt]] = AET_RR3Del_4.mean( axis=1 ).to_numpy()
DictRR4AET1[StatsHdsList[hdrCnt]] = AET_RR4Del_1.mean( axis=1 ).to_numpy()
DictRR4AET2[StatsHdsList[hdrCnt]] = AET_RR4Del_2.mean( axis=1 ).to_numpy()
DictRR4AET3[StatsHdsList[hdrCnt]] = AET_RR4Del_3.mean( axis=1 ).to_numpy()
DictRR4AET4[StatsHdsList[hdrCnt]] = AET_RR4Del_4.mean( axis=1 ).to_numpy()
DictRR5AET1[StatsHdsList[hdrCnt]] = AET_RR5Del_1.mean( axis=1 ).to_numpy()
DictRR5AET2[StatsHdsList[hdrCnt]] = AET_RR5Del_2.mean( axis=1 ).to_numpy()
DictRR5AET3[StatsHdsList[hdrCnt]] = AET_RR5Del_3.mean( axis=1 ).to_numpy()
DictRR5AET4[StatsHdsList[hdrCnt]] = AET_RR5Del_4.mean( axis=1 ).to_numpy()
DictHR1RO1[StatsHdsList[hdrCnt]] = RO_HR1Del_1.mean( axis=1 ).to_numpy()
DictHR1RO2[StatsHdsList[hdrCnt]] = RO_HR1Del_2.mean( axis=1 ).to_numpy()
DictHR1RO3[StatsHdsList[hdrCnt]] = RO_HR1Del_3.mean( axis=1 ).to_numpy()
DictHR1RO4[StatsHdsList[hdrCnt]] = RO_HR1Del_4.mean( axis=1 ).to_numpy()
DictHR2RO1[StatsHdsList[hdrCnt]] = RO_HR2Del_1.mean( axis=1 ).to_numpy()
DictHR2RO2[StatsHdsList[hdrCnt]] = RO_HR2Del_2.mean( axis=1 ).to_numpy()
DictHR2RO3[StatsHdsList[hdrCnt]] = RO_HR2Del_3.mean( axis=1 ).to_numpy()
DictHR2RO4[StatsHdsList[hdrCnt]] = RO_HR2Del_4.mean( axis=1 ).to_numpy()
DictRR1RO1[StatsHdsList[hdrCnt]] = RO_RR1Del_1.mean( axis=1 ).to_numpy()
DictRR1RO2[StatsHdsList[hdrCnt]] = RO_RR1Del_2.mean( axis=1 ).to_numpy()
DictRR1RO3[StatsHdsList[hdrCnt]] = RO_RR1Del_3.mean( axis=1 ).to_numpy()
DictRR1RO4[StatsHdsList[hdrCnt]] = RO_RR1Del_4.mean( axis=1 ).to_numpy()
DictRR2RO1[StatsHdsList[hdrCnt]] = RO_RR2Del_1.mean( axis=1 ).to_numpy()
DictRR2RO2[StatsHdsList[hdrCnt]] = RO_RR2Del_2.mean( axis=1 ).to_numpy()
DictRR2RO3[StatsHdsList[hdrCnt]] = RO_RR2Del_3.mean( axis=1 ).to_numpy()
DictRR2RO4[StatsHdsList[hdrCnt]] = RO_RR2Del_4.mean( axis=1 ).to_numpy()
DictRR3RO1[StatsHdsList[hdrCnt]] = RO_RR3Del_1.mean( axis=1 ).to_numpy()
DictRR3RO2[StatsHdsList[hdrCnt]] = RO_RR3Del_2.mean( axis=1 ).to_numpy()
DictRR3RO3[StatsHdsList[hdrCnt]] = RO_RR3Del_3.mean( axis=1 ).to_numpy()
DictRR3RO4[StatsHdsList[hdrCnt]] = RO_RR3Del_4.mean( axis=1 ).to_numpy()
DictRR4RO1[StatsHdsList[hdrCnt]] = RO_RR4Del_1.mean( axis=1 ).to_numpy()
DictRR4RO2[StatsHdsList[hdrCnt]] = RO_RR4Del_2.mean( axis=1 ).to_numpy()
DictRR4RO3[StatsHdsList[hdrCnt]] = RO_RR4Del_3.mean( axis=1 ).to_numpy()
DictRR4RO4[StatsHdsList[hdrCnt]] = RO_RR4Del_4.mean( axis=1 ).to_numpy()
DictRR5RO1[StatsHdsList[hdrCnt]] = RO_RR5Del_1.mean( axis=1 ).to_numpy()
DictRR5RO2[StatsHdsList[hdrCnt]] = RO_RR5Del_2.mean( axis=1 ).to_numpy()
DictRR5RO3[StatsHdsList[hdrCnt]] = RO_RR5Del_3.mean( axis=1 ).to_numpy()
DictRR5RO4[StatsHdsList[hdrCnt]] = RO_RR5Del_4.mean( axis=1 ).to_numpy()
DictHR1RE1[StatsHdsList[hdrCnt]] = Re_HR1Del_1.mean( axis=1 ).to_numpy()
DictHR1RE2[StatsHdsList[hdrCnt]] = Re_HR1Del_2.mean( axis=1 ).to_numpy()
DictHR1RE3[StatsHdsList[hdrCnt]] = Re_HR1Del_3.mean( axis=1 ).to_numpy()
DictHR1RE4[StatsHdsList[hdrCnt]] = Re_HR1Del_4.mean( axis=1 ).to_numpy()
DictHR2RE1[StatsHdsList[hdrCnt]] = Re_HR2Del_1.mean( axis=1 ).to_numpy()
DictHR2RE2[StatsHdsList[hdrCnt]] = Re_HR2Del_2.mean( axis=1 ).to_numpy()
DictHR2RE3[StatsHdsList[hdrCnt]] = Re_HR2Del_3.mean( axis=1 ).to_numpy()
DictHR2RE4[StatsHdsList[hdrCnt]] = Re_HR2Del_4.mean( axis=1 ).to_numpy()
DictRR1RE1[StatsHdsList[hdrCnt]] = Re_RR1Del_1.mean( axis=1 ).to_numpy()
DictRR1RE2[StatsHdsList[hdrCnt]] = Re_RR1Del_2.mean( axis=1 ).to_numpy()
DictRR1RE3[StatsHdsList[hdrCnt]] = Re_RR1Del_3.mean( axis=1 ).to_numpy()
DictRR1RE4[StatsHdsList[hdrCnt]] = Re_RR1Del_4.mean( axis=1 ).to_numpy()
DictRR2RE1[StatsHdsList[hdrCnt]] = Re_RR2Del_1.mean( axis=1 ).to_numpy()
DictRR2RE2[StatsHdsList[hdrCnt]] = Re_RR2Del_2.mean( axis=1 ).to_numpy()
DictRR2RE3[StatsHdsList[hdrCnt]] = Re_RR2Del_3.mean( axis=1 ).to_numpy()
DictRR2RE4[StatsHdsList[hdrCnt]] = Re_RR2Del_4.mean( axis=1 ).to_numpy()
DictRR3RE1[StatsHdsList[hdrCnt]] = Re_RR3Del_1.mean( axis=1 ).to_numpy()
DictRR3RE2[StatsHdsList[hdrCnt]] = Re_RR3Del_2.mean( axis=1 ).to_numpy()
DictRR3RE3[StatsHdsList[hdrCnt]] = Re_RR3Del_3.mean( axis=1 ).to_numpy()
DictRR3RE4[StatsHdsList[hdrCnt]] = Re_RR3Del_4.mean( axis=1 ).to_numpy()
DictRR4RE1[StatsHdsList[hdrCnt]] = Re_RR4Del_1.mean( axis=1 ).to_numpy()
DictRR4RE2[StatsHdsList[hdrCnt]] = Re_RR4Del_2.mean( axis=1 ).to_numpy()
DictRR4RE3[StatsHdsList[hdrCnt]] = Re_RR4Del_3.mean( axis=1 ).to_numpy()
DictRR4RE4[StatsHdsList[hdrCnt]] = Re_RR4Del_4.mean( axis=1 ).to_numpy()
hdrCnt += 1

In [16]:
DictHR1AET1[StatsHdsList[hdrCnt]] = AET_HR1Del_1.var( axis=1 ).to_numpy()
DictHR1AET2[StatsHdsList[hdrCnt]] = AET_HR1Del_2.var( axis=1 ).to_numpy()
DictHR1AET3[StatsHdsList[hdrCnt]] = AET_HR1Del_3.var( axis=1 ).to_numpy()
DictHR1AET4[StatsHdsList[hdrCnt]] = AET_HR1Del_4.var( axis=1 ).to_numpy()
DictHR2AET1[StatsHdsList[hdrCnt]] = AET_HR2Del_1.var( axis=1 ).to_numpy()
DictHR2AET2[StatsHdsList[hdrCnt]] = AET_HR2Del_2.var( axis=1 ).to_numpy()
DictHR2AET3[StatsHdsList[hdrCnt]] = AET_HR2Del_3.var( axis=1 ).to_numpy()
DictHR2AET4[StatsHdsList[hdrCnt]] = AET_HR2Del_4.var( axis=1 ).to_numpy()
DictRR1AET1[StatsHdsList[hdrCnt]] = AET_RR1Del_1.var( axis=1 ).to_numpy()
DictRR1AET2[StatsHdsList[hdrCnt]] = AET_RR1Del_2.var( axis=1 ).to_numpy()
DictRR1AET3[StatsHdsList[hdrCnt]] = AET_RR1Del_3.var( axis=1 ).to_numpy()
DictRR1AET4[StatsHdsList[hdrCnt]] = AET_RR1Del_4.var( axis=1 ).to_numpy()
DictRR2AET1[StatsHdsList[hdrCnt]] = AET_RR2Del_1.var( axis=1 ).to_numpy()
DictRR2AET2[StatsHdsList[hdrCnt]] = AET_RR2Del_2.var( axis=1 ).to_numpy()
DictRR2AET3[StatsHdsList[hdrCnt]] = AET_RR2Del_3.var( axis=1 ).to_numpy()
DictRR2AET4[StatsHdsList[hdrCnt]] = AET_RR2Del_4.var( axis=1 ).to_numpy()
DictRR3AET1[StatsHdsList[hdrCnt]] = AET_RR3Del_1.var( axis=1 ).to_numpy()
DictRR3AET2[StatsHdsList[hdrCnt]] = AET_RR3Del_2.var( axis=1 ).to_numpy()
DictRR3AET3[StatsHdsList[hdrCnt]] = AET_RR3Del_3.var( axis=1 ).to_numpy()
DictRR3AET4[StatsHdsList[hdrCnt]] = AET_RR3Del_4.var( axis=1 ).to_numpy()
DictRR4AET1[StatsHdsList[hdrCnt]] = AET_RR4Del_1.var( axis=1 ).to_numpy()
DictRR4AET2[StatsHdsList[hdrCnt]] = AET_RR4Del_2.var( axis=1 ).to_numpy()
DictRR4AET3[StatsHdsList[hdrCnt]] = AET_RR4Del_3.var( axis=1 ).to_numpy()
DictRR4AET4[StatsHdsList[hdrCnt]] = AET_RR4Del_4.var( axis=1 ).to_numpy()
DictRR5AET1[StatsHdsList[hdrCnt]] = AET_RR5Del_1.var( axis=1 ).to_numpy()
DictRR5AET2[StatsHdsList[hdrCnt]] = AET_RR5Del_2.var( axis=1 ).to_numpy()
DictRR5AET3[StatsHdsList[hdrCnt]] = AET_RR5Del_3.var( axis=1 ).to_numpy()
DictRR5AET4[StatsHdsList[hdrCnt]] = AET_RR5Del_4.var( axis=1 ).to_numpy()
DictHR1RO1[StatsHdsList[hdrCnt]] = RO_HR1Del_1.var( axis=1 ).to_numpy()
DictHR1RO2[StatsHdsList[hdrCnt]] = RO_HR1Del_2.var( axis=1 ).to_numpy()
DictHR1RO3[StatsHdsList[hdrCnt]] = RO_HR1Del_3.var( axis=1 ).to_numpy()
DictHR1RO4[StatsHdsList[hdrCnt]] = RO_HR1Del_4.var( axis=1 ).to_numpy()
DictHR2RO1[StatsHdsList[hdrCnt]] = RO_HR2Del_1.var( axis=1 ).to_numpy()
DictHR2RO2[StatsHdsList[hdrCnt]] = RO_HR2Del_2.var( axis=1 ).to_numpy()
DictHR2RO3[StatsHdsList[hdrCnt]] = RO_HR2Del_3.var( axis=1 ).to_numpy()
DictHR2RO4[StatsHdsList[hdrCnt]] = RO_HR2Del_4.var( axis=1 ).to_numpy()
DictRR1RO1[StatsHdsList[hdrCnt]] = RO_RR1Del_1.var( axis=1 ).to_numpy()
DictRR1RO2[StatsHdsList[hdrCnt]] = RO_RR1Del_2.var( axis=1 ).to_numpy()
DictRR1RO3[StatsHdsList[hdrCnt]] = RO_RR1Del_3.var( axis=1 ).to_numpy()
DictRR1RO4[StatsHdsList[hdrCnt]] = RO_RR1Del_4.var( axis=1 ).to_numpy()
DictRR2RO1[StatsHdsList[hdrCnt]] = RO_RR2Del_1.var( axis=1 ).to_numpy()
DictRR2RO2[StatsHdsList[hdrCnt]] = RO_RR2Del_2.var( axis=1 ).to_numpy()
DictRR2RO3[StatsHdsList[hdrCnt]] = RO_RR2Del_3.var( axis=1 ).to_numpy()
DictRR2RO4[StatsHdsList[hdrCnt]] = RO_RR2Del_4.var( axis=1 ).to_numpy()
DictRR3RO1[StatsHdsList[hdrCnt]] = RO_RR3Del_1.var( axis=1 ).to_numpy()
DictRR3RO2[StatsHdsList[hdrCnt]] = RO_RR3Del_2.var( axis=1 ).to_numpy()
DictRR3RO3[StatsHdsList[hdrCnt]] = RO_RR3Del_3.var( axis=1 ).to_numpy()
DictRR3RO4[StatsHdsList[hdrCnt]] = RO_RR3Del_4.var( axis=1 ).to_numpy()
DictRR4RO1[StatsHdsList[hdrCnt]] = RO_RR4Del_1.var( axis=1 ).to_numpy()
DictRR4RO2[StatsHdsList[hdrCnt]] = RO_RR4Del_2.var( axis=1 ).to_numpy()
DictRR4RO3[StatsHdsList[hdrCnt]] = RO_RR4Del_3.var( axis=1 ).to_numpy()
DictRR4RO4[StatsHdsList[hdrCnt]] = RO_RR4Del_4.var( axis=1 ).to_numpy()
DictRR5RO1[StatsHdsList[hdrCnt]] = RO_RR5Del_1.var( axis=1 ).to_numpy()
DictRR5RO2[StatsHdsList[hdrCnt]] = RO_RR5Del_2.var( axis=1 ).to_numpy()
DictRR5RO3[StatsHdsList[hdrCnt]] = RO_RR5Del_3.var( axis=1 ).to_numpy()
DictRR5RO4[StatsHdsList[hdrCnt]] = RO_RR5Del_4.var( axis=1 ).to_numpy()
DictHR1RE1[StatsHdsList[hdrCnt]] = Re_HR1Del_1.var( axis=1 ).to_numpy()
DictHR1RE2[StatsHdsList[hdrCnt]] = Re_HR1Del_2.var( axis=1 ).to_numpy()
DictHR1RE3[StatsHdsList[hdrCnt]] = Re_HR1Del_3.var( axis=1 ).to_numpy()
DictHR1RE4[StatsHdsList[hdrCnt]] = Re_HR1Del_4.var( axis=1 ).to_numpy()
DictHR2RE1[StatsHdsList[hdrCnt]] = Re_HR2Del_1.var( axis=1 ).to_numpy()
DictHR2RE2[StatsHdsList[hdrCnt]] = Re_HR2Del_2.var( axis=1 ).to_numpy()
DictHR2RE3[StatsHdsList[hdrCnt]] = Re_HR2Del_3.var( axis=1 ).to_numpy()
DictHR2RE4[StatsHdsList[hdrCnt]] = Re_HR2Del_4.var( axis=1 ).to_numpy()
DictRR1RE1[StatsHdsList[hdrCnt]] = Re_RR1Del_1.var( axis=1 ).to_numpy()
DictRR1RE2[StatsHdsList[hdrCnt]] = Re_RR1Del_2.var( axis=1 ).to_numpy()
DictRR1RE3[StatsHdsList[hdrCnt]] = Re_RR1Del_3.var( axis=1 ).to_numpy()
DictRR1RE4[StatsHdsList[hdrCnt]] = Re_RR1Del_4.var( axis=1 ).to_numpy()
DictRR2RE1[StatsHdsList[hdrCnt]] = Re_RR2Del_1.var( axis=1 ).to_numpy()
DictRR2RE2[StatsHdsList[hdrCnt]] = Re_RR2Del_2.var( axis=1 ).to_numpy()
DictRR2RE3[StatsHdsList[hdrCnt]] = Re_RR2Del_3.var( axis=1 ).to_numpy()
DictRR2RE4[StatsHdsList[hdrCnt]] = Re_RR2Del_4.var( axis=1 ).to_numpy()
DictRR3RE1[StatsHdsList[hdrCnt]] = Re_RR3Del_1.var( axis=1 ).to_numpy()
DictRR3RE2[StatsHdsList[hdrCnt]] = Re_RR3Del_2.var( axis=1 ).to_numpy()
DictRR3RE3[StatsHdsList[hdrCnt]] = Re_RR3Del_3.var( axis=1 ).to_numpy()
DictRR3RE4[StatsHdsList[hdrCnt]] = Re_RR3Del_4.var( axis=1 ).to_numpy()
DictRR4RE1[StatsHdsList[hdrCnt]] = Re_RR4Del_1.var( axis=1 ).to_numpy()
DictRR4RE2[StatsHdsList[hdrCnt]] = Re_RR4Del_2.var( axis=1 ).to_numpy()
DictRR4RE3[StatsHdsList[hdrCnt]] = Re_RR4Del_3.var( axis=1 ).to_numpy()
DictRR4RE4[StatsHdsList[hdrCnt]] = Re_RR4Del_4.var( axis=1 ).to_numpy()
hdrCnt += 1

In [17]:
DictHR1AET1[StatsHdsList[hdrCnt]] = AET_HR1Del_1.skew( axis=1 ).to_numpy()
DictHR1AET2[StatsHdsList[hdrCnt]] = AET_HR1Del_2.skew( axis=1 ).to_numpy()
DictHR1AET3[StatsHdsList[hdrCnt]] = AET_HR1Del_3.skew( axis=1 ).to_numpy()
DictHR1AET4[StatsHdsList[hdrCnt]] = AET_HR1Del_4.skew( axis=1 ).to_numpy()
DictHR2AET1[StatsHdsList[hdrCnt]] = AET_HR2Del_1.skew( axis=1 ).to_numpy()
DictHR2AET2[StatsHdsList[hdrCnt]] = AET_HR2Del_2.skew( axis=1 ).to_numpy()
DictHR2AET3[StatsHdsList[hdrCnt]] = AET_HR2Del_3.skew( axis=1 ).to_numpy()
DictHR2AET4[StatsHdsList[hdrCnt]] = AET_HR2Del_4.skew( axis=1 ).to_numpy()
DictRR1AET1[StatsHdsList[hdrCnt]] = AET_RR1Del_1.skew( axis=1 ).to_numpy()
DictRR1AET2[StatsHdsList[hdrCnt]] = AET_RR1Del_2.skew( axis=1 ).to_numpy()
DictRR1AET3[StatsHdsList[hdrCnt]] = AET_RR1Del_3.skew( axis=1 ).to_numpy()
DictRR1AET4[StatsHdsList[hdrCnt]] = AET_RR1Del_4.skew( axis=1 ).to_numpy()
DictRR2AET1[StatsHdsList[hdrCnt]] = AET_RR2Del_1.skew( axis=1 ).to_numpy()
DictRR2AET2[StatsHdsList[hdrCnt]] = AET_RR2Del_2.skew( axis=1 ).to_numpy()
DictRR2AET3[StatsHdsList[hdrCnt]] = AET_RR2Del_3.skew( axis=1 ).to_numpy()
DictRR2AET4[StatsHdsList[hdrCnt]] = AET_RR2Del_4.skew( axis=1 ).to_numpy()
DictRR3AET1[StatsHdsList[hdrCnt]] = AET_RR3Del_1.skew( axis=1 ).to_numpy()
DictRR3AET2[StatsHdsList[hdrCnt]] = AET_RR3Del_2.skew( axis=1 ).to_numpy()
DictRR3AET3[StatsHdsList[hdrCnt]] = AET_RR3Del_3.skew( axis=1 ).to_numpy()
DictRR3AET4[StatsHdsList[hdrCnt]] = AET_RR3Del_4.skew( axis=1 ).to_numpy()
DictRR4AET1[StatsHdsList[hdrCnt]] = AET_RR4Del_1.skew( axis=1 ).to_numpy()
DictRR4AET2[StatsHdsList[hdrCnt]] = AET_RR4Del_2.skew( axis=1 ).to_numpy()
DictRR4AET3[StatsHdsList[hdrCnt]] = AET_RR4Del_3.skew( axis=1 ).to_numpy()
DictRR4AET4[StatsHdsList[hdrCnt]] = AET_RR4Del_4.skew( axis=1 ).to_numpy()
DictRR5AET1[StatsHdsList[hdrCnt]] = AET_RR5Del_1.skew( axis=1 ).to_numpy()
DictRR5AET2[StatsHdsList[hdrCnt]] = AET_RR5Del_2.skew( axis=1 ).to_numpy()
DictRR5AET3[StatsHdsList[hdrCnt]] = AET_RR5Del_3.skew( axis=1 ).to_numpy()
DictRR5AET4[StatsHdsList[hdrCnt]] = AET_RR5Del_4.skew( axis=1 ).to_numpy()
DictHR1RO1[StatsHdsList[hdrCnt]] = RO_HR1Del_1.skew( axis=1 ).to_numpy()
DictHR1RO2[StatsHdsList[hdrCnt]] = RO_HR1Del_2.skew( axis=1 ).to_numpy()
DictHR1RO3[StatsHdsList[hdrCnt]] = RO_HR1Del_3.skew( axis=1 ).to_numpy()
DictHR1RO4[StatsHdsList[hdrCnt]] = RO_HR1Del_4.skew( axis=1 ).to_numpy()
DictHR2RO1[StatsHdsList[hdrCnt]] = RO_HR2Del_1.skew( axis=1 ).to_numpy()
DictHR2RO2[StatsHdsList[hdrCnt]] = RO_HR2Del_2.skew( axis=1 ).to_numpy()
DictHR2RO3[StatsHdsList[hdrCnt]] = RO_HR2Del_3.skew( axis=1 ).to_numpy()
DictHR2RO4[StatsHdsList[hdrCnt]] = RO_HR2Del_4.skew( axis=1 ).to_numpy()
DictRR1RO1[StatsHdsList[hdrCnt]] = RO_RR1Del_1.skew( axis=1 ).to_numpy()
DictRR1RO2[StatsHdsList[hdrCnt]] = RO_RR1Del_2.skew( axis=1 ).to_numpy()
DictRR1RO3[StatsHdsList[hdrCnt]] = RO_RR1Del_3.skew( axis=1 ).to_numpy()
DictRR1RO4[StatsHdsList[hdrCnt]] = RO_RR1Del_4.skew( axis=1 ).to_numpy()
DictRR2RO1[StatsHdsList[hdrCnt]] = RO_RR2Del_1.skew( axis=1 ).to_numpy()
DictRR2RO2[StatsHdsList[hdrCnt]] = RO_RR2Del_2.skew( axis=1 ).to_numpy()
DictRR2RO3[StatsHdsList[hdrCnt]] = RO_RR2Del_3.skew( axis=1 ).to_numpy()
DictRR2RO4[StatsHdsList[hdrCnt]] = RO_RR2Del_4.skew( axis=1 ).to_numpy()
DictRR3RO1[StatsHdsList[hdrCnt]] = RO_RR3Del_1.skew( axis=1 ).to_numpy()
DictRR3RO2[StatsHdsList[hdrCnt]] = RO_RR3Del_2.skew( axis=1 ).to_numpy()
DictRR3RO3[StatsHdsList[hdrCnt]] = RO_RR3Del_3.skew( axis=1 ).to_numpy()
DictRR3RO4[StatsHdsList[hdrCnt]] = RO_RR3Del_4.skew( axis=1 ).to_numpy()
DictRR4RO1[StatsHdsList[hdrCnt]] = RO_RR4Del_1.skew( axis=1 ).to_numpy()
DictRR4RO2[StatsHdsList[hdrCnt]] = RO_RR4Del_2.skew( axis=1 ).to_numpy()
DictRR4RO3[StatsHdsList[hdrCnt]] = RO_RR4Del_3.skew( axis=1 ).to_numpy()
DictRR4RO4[StatsHdsList[hdrCnt]] = RO_RR4Del_4.skew( axis=1 ).to_numpy()
DictRR5RO1[StatsHdsList[hdrCnt]] = RO_RR5Del_1.skew( axis=1 ).to_numpy()
DictRR5RO2[StatsHdsList[hdrCnt]] = RO_RR5Del_2.skew( axis=1 ).to_numpy()
DictRR5RO3[StatsHdsList[hdrCnt]] = RO_RR5Del_3.skew( axis=1 ).to_numpy()
DictRR5RO4[StatsHdsList[hdrCnt]] = RO_RR5Del_4.skew( axis=1 ).to_numpy()
DictHR1RE1[StatsHdsList[hdrCnt]] = Re_HR1Del_1.skew( axis=1 ).to_numpy()
DictHR1RE2[StatsHdsList[hdrCnt]] = Re_HR1Del_2.skew( axis=1 ).to_numpy()
DictHR1RE3[StatsHdsList[hdrCnt]] = Re_HR1Del_3.skew( axis=1 ).to_numpy()
DictHR1RE4[StatsHdsList[hdrCnt]] = Re_HR1Del_4.skew( axis=1 ).to_numpy()
DictHR2RE1[StatsHdsList[hdrCnt]] = Re_HR2Del_1.skew( axis=1 ).to_numpy()
DictHR2RE2[StatsHdsList[hdrCnt]] = Re_HR2Del_2.skew( axis=1 ).to_numpy()
DictHR2RE3[StatsHdsList[hdrCnt]] = Re_HR2Del_3.skew( axis=1 ).to_numpy()
DictHR2RE4[StatsHdsList[hdrCnt]] = Re_HR2Del_4.skew( axis=1 ).to_numpy()
DictRR1RE1[StatsHdsList[hdrCnt]] = Re_RR1Del_1.skew( axis=1 ).to_numpy()
DictRR1RE2[StatsHdsList[hdrCnt]] = Re_RR1Del_2.skew( axis=1 ).to_numpy()
DictRR1RE3[StatsHdsList[hdrCnt]] = Re_RR1Del_3.skew( axis=1 ).to_numpy()
DictRR1RE4[StatsHdsList[hdrCnt]] = Re_RR1Del_4.skew( axis=1 ).to_numpy()
DictRR2RE1[StatsHdsList[hdrCnt]] = Re_RR2Del_1.skew( axis=1 ).to_numpy()
DictRR2RE2[StatsHdsList[hdrCnt]] = Re_RR2Del_2.skew( axis=1 ).to_numpy()
DictRR2RE3[StatsHdsList[hdrCnt]] = Re_RR2Del_3.skew( axis=1 ).to_numpy()
DictRR2RE4[StatsHdsList[hdrCnt]] = Re_RR2Del_4.skew( axis=1 ).to_numpy()
DictRR3RE1[StatsHdsList[hdrCnt]] = Re_RR3Del_1.skew( axis=1 ).to_numpy()
DictRR3RE2[StatsHdsList[hdrCnt]] = Re_RR3Del_2.skew( axis=1 ).to_numpy()
DictRR3RE3[StatsHdsList[hdrCnt]] = Re_RR3Del_3.skew( axis=1 ).to_numpy()
DictRR3RE4[StatsHdsList[hdrCnt]] = Re_RR3Del_4.skew( axis=1 ).to_numpy()
DictRR4RE1[StatsHdsList[hdrCnt]] = Re_RR4Del_1.skew( axis=1 ).to_numpy()
DictRR4RE2[StatsHdsList[hdrCnt]] = Re_RR4Del_2.skew( axis=1 ).to_numpy()
DictRR4RE3[StatsHdsList[hdrCnt]] = Re_RR4Del_3.skew( axis=1 ).to_numpy()
DictRR4RE4[StatsHdsList[hdrCnt]] = Re_RR4Del_4.skew( axis=1 ).to_numpy()
hdrCnt += 1

In [18]:
DictHR1AET1[StatsHdsList[hdrCnt]] = AET_HR1Del_1.kurt( axis=1 ).to_numpy()
DictHR1AET2[StatsHdsList[hdrCnt]] = AET_HR1Del_2.kurt( axis=1 ).to_numpy()
DictHR1AET3[StatsHdsList[hdrCnt]] = AET_HR1Del_3.kurt( axis=1 ).to_numpy()
DictHR1AET4[StatsHdsList[hdrCnt]] = AET_HR1Del_4.kurt( axis=1 ).to_numpy()
DictHR2AET1[StatsHdsList[hdrCnt]] = AET_HR2Del_1.kurt( axis=1 ).to_numpy()
DictHR2AET2[StatsHdsList[hdrCnt]] = AET_HR2Del_2.kurt( axis=1 ).to_numpy()
DictHR2AET3[StatsHdsList[hdrCnt]] = AET_HR2Del_3.kurt( axis=1 ).to_numpy()
DictHR2AET4[StatsHdsList[hdrCnt]] = AET_HR2Del_4.kurt( axis=1 ).to_numpy()
DictRR1AET1[StatsHdsList[hdrCnt]] = AET_RR1Del_1.kurt( axis=1 ).to_numpy()
DictRR1AET2[StatsHdsList[hdrCnt]] = AET_RR1Del_2.kurt( axis=1 ).to_numpy()
DictRR1AET3[StatsHdsList[hdrCnt]] = AET_RR1Del_3.kurt( axis=1 ).to_numpy()
DictRR1AET4[StatsHdsList[hdrCnt]] = AET_RR1Del_4.kurt( axis=1 ).to_numpy()
DictRR2AET1[StatsHdsList[hdrCnt]] = AET_RR2Del_1.kurt( axis=1 ).to_numpy()
DictRR2AET2[StatsHdsList[hdrCnt]] = AET_RR2Del_2.kurt( axis=1 ).to_numpy()
DictRR2AET3[StatsHdsList[hdrCnt]] = AET_RR2Del_3.kurt( axis=1 ).to_numpy()
DictRR2AET4[StatsHdsList[hdrCnt]] = AET_RR2Del_4.kurt( axis=1 ).to_numpy()
DictRR3AET1[StatsHdsList[hdrCnt]] = AET_RR3Del_1.kurt( axis=1 ).to_numpy()
DictRR3AET2[StatsHdsList[hdrCnt]] = AET_RR3Del_2.kurt( axis=1 ).to_numpy()
DictRR3AET3[StatsHdsList[hdrCnt]] = AET_RR3Del_3.kurt( axis=1 ).to_numpy()
DictRR3AET4[StatsHdsList[hdrCnt]] = AET_RR3Del_4.kurt( axis=1 ).to_numpy()
DictRR4AET1[StatsHdsList[hdrCnt]] = AET_RR4Del_1.kurt( axis=1 ).to_numpy()
DictRR4AET2[StatsHdsList[hdrCnt]] = AET_RR4Del_2.kurt( axis=1 ).to_numpy()
DictRR4AET3[StatsHdsList[hdrCnt]] = AET_RR4Del_3.kurt( axis=1 ).to_numpy()
DictRR4AET4[StatsHdsList[hdrCnt]] = AET_RR4Del_4.kurt( axis=1 ).to_numpy()
DictRR5AET1[StatsHdsList[hdrCnt]] = AET_RR5Del_1.kurt( axis=1 ).to_numpy()
DictRR5AET2[StatsHdsList[hdrCnt]] = AET_RR5Del_2.kurt( axis=1 ).to_numpy()
DictRR5AET3[StatsHdsList[hdrCnt]] = AET_RR5Del_3.kurt( axis=1 ).to_numpy()
DictRR5AET4[StatsHdsList[hdrCnt]] = AET_RR5Del_4.kurt( axis=1 ).to_numpy()
DictHR1RO1[StatsHdsList[hdrCnt]] = RO_HR1Del_1.kurt( axis=1 ).to_numpy()
DictHR1RO2[StatsHdsList[hdrCnt]] = RO_HR1Del_2.kurt( axis=1 ).to_numpy()
DictHR1RO3[StatsHdsList[hdrCnt]] = RO_HR1Del_3.kurt( axis=1 ).to_numpy()
DictHR1RO4[StatsHdsList[hdrCnt]] = RO_HR1Del_4.kurt( axis=1 ).to_numpy()
DictHR2RO1[StatsHdsList[hdrCnt]] = RO_HR2Del_1.kurt( axis=1 ).to_numpy()
DictHR2RO2[StatsHdsList[hdrCnt]] = RO_HR2Del_2.kurt( axis=1 ).to_numpy()
DictHR2RO3[StatsHdsList[hdrCnt]] = RO_HR2Del_3.kurt( axis=1 ).to_numpy()
DictHR2RO4[StatsHdsList[hdrCnt]] = RO_HR2Del_4.kurt( axis=1 ).to_numpy()
DictRR1RO1[StatsHdsList[hdrCnt]] = RO_RR1Del_1.kurt( axis=1 ).to_numpy()
DictRR1RO2[StatsHdsList[hdrCnt]] = RO_RR1Del_2.kurt( axis=1 ).to_numpy()
DictRR1RO3[StatsHdsList[hdrCnt]] = RO_RR1Del_3.kurt( axis=1 ).to_numpy()
DictRR1RO4[StatsHdsList[hdrCnt]] = RO_RR1Del_4.kurt( axis=1 ).to_numpy()
DictRR2RO1[StatsHdsList[hdrCnt]] = RO_RR2Del_1.kurt( axis=1 ).to_numpy()
DictRR2RO2[StatsHdsList[hdrCnt]] = RO_RR2Del_2.kurt( axis=1 ).to_numpy()
DictRR2RO3[StatsHdsList[hdrCnt]] = RO_RR2Del_3.kurt( axis=1 ).to_numpy()
DictRR2RO4[StatsHdsList[hdrCnt]] = RO_RR2Del_4.kurt( axis=1 ).to_numpy()
DictRR3RO1[StatsHdsList[hdrCnt]] = RO_RR3Del_1.kurt( axis=1 ).to_numpy()
DictRR3RO2[StatsHdsList[hdrCnt]] = RO_RR3Del_2.kurt( axis=1 ).to_numpy()
DictRR3RO3[StatsHdsList[hdrCnt]] = RO_RR3Del_3.kurt( axis=1 ).to_numpy()
DictRR3RO4[StatsHdsList[hdrCnt]] = RO_RR3Del_4.kurt( axis=1 ).to_numpy()
DictRR4RO1[StatsHdsList[hdrCnt]] = RO_RR4Del_1.kurt( axis=1 ).to_numpy()
DictRR4RO2[StatsHdsList[hdrCnt]] = RO_RR4Del_2.kurt( axis=1 ).to_numpy()
DictRR4RO3[StatsHdsList[hdrCnt]] = RO_RR4Del_3.kurt( axis=1 ).to_numpy()
DictRR4RO4[StatsHdsList[hdrCnt]] = RO_RR4Del_4.kurt( axis=1 ).to_numpy()
DictRR5RO1[StatsHdsList[hdrCnt]] = RO_RR5Del_1.kurt( axis=1 ).to_numpy()
DictRR5RO2[StatsHdsList[hdrCnt]] = RO_RR5Del_2.kurt( axis=1 ).to_numpy()
DictRR5RO3[StatsHdsList[hdrCnt]] = RO_RR5Del_3.kurt( axis=1 ).to_numpy()
DictRR5RO4[StatsHdsList[hdrCnt]] = RO_RR5Del_4.kurt( axis=1 ).to_numpy()
DictHR1RE1[StatsHdsList[hdrCnt]] = Re_HR1Del_1.kurt( axis=1 ).to_numpy()
DictHR1RE2[StatsHdsList[hdrCnt]] = Re_HR1Del_2.kurt( axis=1 ).to_numpy()
DictHR1RE3[StatsHdsList[hdrCnt]] = Re_HR1Del_3.kurt( axis=1 ).to_numpy()
DictHR1RE4[StatsHdsList[hdrCnt]] = Re_HR1Del_4.kurt( axis=1 ).to_numpy()
DictHR2RE1[StatsHdsList[hdrCnt]] = Re_HR2Del_1.kurt( axis=1 ).to_numpy()
DictHR2RE2[StatsHdsList[hdrCnt]] = Re_HR2Del_2.kurt( axis=1 ).to_numpy()
DictHR2RE3[StatsHdsList[hdrCnt]] = Re_HR2Del_3.kurt( axis=1 ).to_numpy()
DictHR2RE4[StatsHdsList[hdrCnt]] = Re_HR2Del_4.kurt( axis=1 ).to_numpy()
DictRR1RE1[StatsHdsList[hdrCnt]] = Re_RR1Del_1.kurt( axis=1 ).to_numpy()
DictRR1RE2[StatsHdsList[hdrCnt]] = Re_RR1Del_2.kurt( axis=1 ).to_numpy()
DictRR1RE3[StatsHdsList[hdrCnt]] = Re_RR1Del_3.kurt( axis=1 ).to_numpy()
DictRR1RE4[StatsHdsList[hdrCnt]] = Re_RR1Del_4.kurt( axis=1 ).to_numpy()
DictRR2RE1[StatsHdsList[hdrCnt]] = Re_RR2Del_1.kurt( axis=1 ).to_numpy()
DictRR2RE2[StatsHdsList[hdrCnt]] = Re_RR2Del_2.kurt( axis=1 ).to_numpy()
DictRR2RE3[StatsHdsList[hdrCnt]] = Re_RR2Del_3.kurt( axis=1 ).to_numpy()
DictRR2RE4[StatsHdsList[hdrCnt]] = Re_RR2Del_4.kurt( axis=1 ).to_numpy()
DictRR3RE1[StatsHdsList[hdrCnt]] = Re_RR3Del_1.kurt( axis=1 ).to_numpy()
DictRR3RE2[StatsHdsList[hdrCnt]] = Re_RR3Del_2.kurt( axis=1 ).to_numpy()
DictRR3RE3[StatsHdsList[hdrCnt]] = Re_RR3Del_3.kurt( axis=1 ).to_numpy()
DictRR3RE4[StatsHdsList[hdrCnt]] = Re_RR3Del_4.kurt( axis=1 ).to_numpy()
DictRR4RE1[StatsHdsList[hdrCnt]] = Re_RR4Del_1.kurt( axis=1 ).to_numpy()
DictRR4RE2[StatsHdsList[hdrCnt]] = Re_RR4Del_2.kurt( axis=1 ).to_numpy()
DictRR4RE3[StatsHdsList[hdrCnt]] = Re_RR4Del_3.kurt( axis=1 ).to_numpy()
DictRR4RE4[StatsHdsList[hdrCnt]] = Re_RR4Del_4.kurt( axis=1 ).to_numpy()

Make our DataFrames from the Stats

In [19]:
TimIndex1 = AET_HR1Del_1.index
TimIndex2 = AET_HR1Del_2.index
TimIndex3 = AET_HR1Del_3.index
TimIndex4 = AET_HR1Del_4.index

In [20]:
HR1AETStats1 = pd.DataFrame(index=TimIndex1, data=DictHR1AET1 )
HR1AETStats2 = pd.DataFrame(index=TimIndex2, data=DictHR1AET2 )
HR1AETStats3 = pd.DataFrame(index=TimIndex3, data=DictHR1AET3 )
HR1AETStats4 = pd.DataFrame(index=TimIndex4, data=DictHR1AET4 )
HR2AETStats1 = pd.DataFrame(index=TimIndex1, data=DictHR2AET1 )
HR2AETStats2 = pd.DataFrame(index=TimIndex2, data=DictHR2AET2 )
HR2AETStats3 = pd.DataFrame(index=TimIndex3, data=DictHR2AET3 )
HR2AETStats4 = pd.DataFrame(index=TimIndex4, data=DictHR2AET4 )
RR1AETStats1 = pd.DataFrame(index=TimIndex1, data=DictRR1AET1 )
RR1AETStats2 = pd.DataFrame(index=TimIndex2, data=DictRR1AET2 )
RR1AETStats3 = pd.DataFrame(index=TimIndex3, data=DictRR1AET3 )
RR1AETStats4 = pd.DataFrame(index=TimIndex4, data=DictRR1AET4 )
RR2AETStats1 = pd.DataFrame(index=TimIndex1, data=DictRR2AET1 )
RR2AETStats2 = pd.DataFrame(index=TimIndex2, data=DictRR2AET2 )
RR2AETStats3 = pd.DataFrame(index=TimIndex3, data=DictRR2AET3 )
RR2AETStats4 = pd.DataFrame(index=TimIndex4, data=DictRR2AET4 )
RR3AETStats1 = pd.DataFrame(index=TimIndex1, data=DictRR3AET1 )
RR3AETStats2 = pd.DataFrame(index=TimIndex2, data=DictRR3AET2 )
RR3AETStats3 = pd.DataFrame(index=TimIndex3, data=DictRR3AET3 )
RR3AETStats4 = pd.DataFrame(index=TimIndex4, data=DictRR3AET4 )
RR4AETStats1 = pd.DataFrame(index=TimIndex1, data=DictRR4AET1 )
RR4AETStats2 = pd.DataFrame(index=TimIndex2, data=DictRR4AET2 )
RR4AETStats3 = pd.DataFrame(index=TimIndex3, data=DictRR4AET3 )
RR4AETStats4 = pd.DataFrame(index=TimIndex4, data=DictRR4AET4 )
RR5AETStats1 = pd.DataFrame(index=TimIndex1, data=DictRR5AET1 )
RR5AETStats2 = pd.DataFrame(index=TimIndex2, data=DictRR5AET2 )
RR5AETStats3 = pd.DataFrame(index=TimIndex3, data=DictRR5AET3 )
RR5AETStats4 = pd.DataFrame(index=TimIndex4, data=DictRR5AET4 )

In [21]:
HR1ROStats1 = pd.DataFrame(index=TimIndex1, data=DictHR1RO1 )
HR1ROStats2 = pd.DataFrame(index=TimIndex2, data=DictHR1RO2 )
HR1ROStats3 = pd.DataFrame(index=TimIndex3, data=DictHR1RO3 )
HR1ROStats4 = pd.DataFrame(index=TimIndex4, data=DictHR1RO4 )
HR2ROStats1 = pd.DataFrame(index=TimIndex1, data=DictHR2RO1 )
HR2ROStats2 = pd.DataFrame(index=TimIndex2, data=DictHR2RO2 )
HR2ROStats3 = pd.DataFrame(index=TimIndex3, data=DictHR2RO3 )
HR2ROStats4 = pd.DataFrame(index=TimIndex4, data=DictHR2RO4 )
RR1ROStats1 = pd.DataFrame(index=TimIndex1, data=DictRR1RO1 )
RR1ROStats2 = pd.DataFrame(index=TimIndex2, data=DictRR1RO2 )
RR1ROStats3 = pd.DataFrame(index=TimIndex3, data=DictRR1RO3 )
RR1ROStats4 = pd.DataFrame(index=TimIndex4, data=DictRR1RO4 )
RR2ROStats1 = pd.DataFrame(index=TimIndex1, data=DictRR2RO1 )
RR2ROStats2 = pd.DataFrame(index=TimIndex2, data=DictRR2RO2 )
RR2ROStats3 = pd.DataFrame(index=TimIndex3, data=DictRR2RO3 )
RR2ROStats4 = pd.DataFrame(index=TimIndex4, data=DictRR2RO4 )
RR3ROStats1 = pd.DataFrame(index=TimIndex1, data=DictRR3RO1 )
RR3ROStats2 = pd.DataFrame(index=TimIndex2, data=DictRR3RO2 )
RR3ROStats3 = pd.DataFrame(index=TimIndex3, data=DictRR3RO3 )
RR3ROStats4 = pd.DataFrame(index=TimIndex4, data=DictRR3RO4 )
RR4ROStats1 = pd.DataFrame(index=TimIndex1, data=DictRR4RO1 )
RR4ROStats2 = pd.DataFrame(index=TimIndex2, data=DictRR4RO2 )
RR4ROStats3 = pd.DataFrame(index=TimIndex3, data=DictRR4RO3 )
RR4ROStats4 = pd.DataFrame(index=TimIndex4, data=DictRR4RO4 )
RR5ROStats1 = pd.DataFrame(index=TimIndex1, data=DictRR5RO1 )
RR5ROStats2 = pd.DataFrame(index=TimIndex2, data=DictRR5RO2 )
RR5ROStats3 = pd.DataFrame(index=TimIndex3, data=DictRR5RO3 )
RR5ROStats4 = pd.DataFrame(index=TimIndex4, data=DictRR5RO4 )

In [22]:
HR1REStats1 = pd.DataFrame(index=TimIndex1, data=DictHR1RE1 )
HR1REStats2 = pd.DataFrame(index=TimIndex2, data=DictHR1RE2 )
HR1REStats3 = pd.DataFrame(index=TimIndex3, data=DictHR1RE3 )
HR1REStats4 = pd.DataFrame(index=TimIndex4, data=DictHR1RE4 )
HR2REStats1 = pd.DataFrame(index=TimIndex1, data=DictHR2RE1 )
HR2REStats2 = pd.DataFrame(index=TimIndex2, data=DictHR2RE2 )
HR2REStats3 = pd.DataFrame(index=TimIndex3, data=DictHR2RE3 )
HR2REStats4 = pd.DataFrame(index=TimIndex4, data=DictHR2RE4 )
RR1REStats1 = pd.DataFrame(index=TimIndex1, data=DictRR1RE1 )
RR1REStats2 = pd.DataFrame(index=TimIndex2, data=DictRR1RE2 )
RR1REStats3 = pd.DataFrame(index=TimIndex3, data=DictRR1RE3 )
RR1REStats4 = pd.DataFrame(index=TimIndex4, data=DictRR1RE4 )
RR2REStats1 = pd.DataFrame(index=TimIndex1, data=DictRR2RE1 )
RR2REStats2 = pd.DataFrame(index=TimIndex2, data=DictRR2RE2 )
RR2REStats3 = pd.DataFrame(index=TimIndex3, data=DictRR2RE3 )
RR2REStats4 = pd.DataFrame(index=TimIndex4, data=DictRR2RE4 )
RR3REStats1 = pd.DataFrame(index=TimIndex1, data=DictRR3RE1 )
RR3REStats2 = pd.DataFrame(index=TimIndex2, data=DictRR3RE2 )
RR3REStats3 = pd.DataFrame(index=TimIndex3, data=DictRR3RE3 )
RR3REStats4 = pd.DataFrame(index=TimIndex4, data=DictRR3RE4 )
RR4REStats1 = pd.DataFrame(index=TimIndex1, data=DictRR4RE1 )
RR4REStats2 = pd.DataFrame(index=TimIndex2, data=DictRR4RE2 )
RR4REStats3 = pd.DataFrame(index=TimIndex3, data=DictRR4RE3 )
RR4REStats4 = pd.DataFrame(index=TimIndex4, data=DictRR4RE4 )

Output to a spreadsheets

In [23]:
OutFileName = "Del_HR1_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    HR1AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    HR1AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    HR1AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    HR1AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    HR1ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR1ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR1ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR1ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR1REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR1REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR1REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR1REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [24]:
OutFileName = "Del_HR2_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    HR2AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    HR2AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    HR2AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    HR2AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    HR2ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR2ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR2ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR2ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR2REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR2REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR2REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    HR2REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [25]:
OutFileName = "Del_RR1_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    RR1AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR1AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR1AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR1AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR1ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR1ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR1ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR1ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR1REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR1REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR1REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR1REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [26]:
OutFileName = "Del_RR2_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    RR2AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR2AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR2AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR2AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR2ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR2ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR2ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR2ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR2REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR2REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR2REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR2REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [27]:
OutFileName = "Del_RR3_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    RR3AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR3AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR3AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR3AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR3ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR3ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR3ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR3ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR3REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR3REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR3REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR3REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [28]:
OutFileName = "Del_RR4_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    RR4AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR4AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR4AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR4AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR4ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR4ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR4ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR4ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR4REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR4REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR4REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR4REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [29]:
OutFileName = "Del_RR5_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    RR5AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR5AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR5AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR5AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                           columns=StatsHdsList, na_rep=str(np.nan) )
    RR5ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR5ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR5ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
    RR5ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                          columns=StatsHdsList, na_rep=str(np.nan) )
# end with